In [ ]:
import os
import pandas as pd
import numpy as np

# ====== CONFIG ======
BASE = "/content"
PROVIDERS = {
    "carris":               os.path.join(BASE, "carris"),
    "cp":                   os.path.join(BASE, "cp"),
    "fertagus":             os.path.join(BASE, "fertagus"),
    "metro":                os.path.join(BASE, "metro"),
    "mst":                  os.path.join(BASE, "mst"),
    "transportes_barreiro": os.path.join(BASE, "transportes_barreiro"),
    "transtejo":            os.path.join(BASE, "transtejo"),
}

# ====== Helpers ======
def _read_any_sep(path, usecols=None):
    try:
        df = pd.read_csv(path, dtype=str, engine="python", sep=None,
                         on_bad_lines="skip",  usecols=usecols)
    except Exception:
        df = pd.read_csv(path, dtype=str, engine="python", sep=";",
                         on_bad_lines="skip",  usecols=usecols)
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def _pick_first(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _clean_stops(df, feed):
    stop_id_col   = _pick_first(df, ["stop_id", "id", "stopcode"])
    stop_name_col = _pick_first(df, ["stop_name", "stop_name_new", "name", "stopdesc"])
    lat_col       = _pick_first(df, ["stop_lat", "lat", "latitude"])
    lon_col       = _pick_first(df, ["stop_lon", "lon", "longitude", "long"])

    out = pd.DataFrame()
    out["stop_id"]   = df[stop_id_col].astype(str) if stop_id_col else pd.Series(dtype="object")
    out["stop_name"] = df[stop_name_col].astype(str) if stop_name_col else pd.Series(dtype="object")
    out["stop_lat"]  = pd.to_numeric(df[lat_col], errors="coerce") if lat_col else pd.Series(dtype="float64")
    out["stop_lon"]  = pd.to_numeric(df[lon_col], errors="coerce") if lon_col else pd.Series(dtype="float64")
    out["feed"]      = feed

    # ---- sid (namespaced stop id)
    out["sid"] = (out["feed"].astype(str) + "_" + out["stop_id"].fillna("")).str.strip()

    # enforce order
    return out[["sid","stop_id","stop_name","stop_lat","stop_lon","feed"]]

def _clean_stop_times(df, feed):
    trip_col = _pick_first(df, ["trip_id", "trip"])
    arr_col  = _pick_first(df, ["arrival_time", "arrival"])
    dep_col  = _pick_first(df, ["departure_time", "departure"])
    stop_col = _pick_first(df, ["stop_id", "stop"])
    seq_col  = _pick_first(df, ["stop_sequence", "sequence", "stopseq", "stop_order"])

    out = pd.DataFrame()
    out["trip_id"]        = df[trip_col].astype(str) if trip_col else pd.Series(dtype="object")
    out["arrival_time"]   = df[arr_col].astype(str).str.strip() if arr_col else pd.Series(dtype="object")
    out["departure_time"] = df[dep_col].astype(str).str.strip() if dep_col else pd.Series(dtype="object")
    out["stop_id"]        = df[stop_col].astype(str) if stop_col else pd.Series(dtype="object")
    out["stop_sequence"]  = (
        df[seq_col].astype(str).str.extract(r'(-?\d+)', expand=False).astype("Int64")
        if seq_col else pd.Series(pd.array([pd.NA]*len(df), dtype="Int64"))
    )
    out["feed"] = feed

    # ---- sid for stop_times, same construction
    out["sid"] = (out["feed"].astype(str) + "_" + out["stop_id"].fillna("")).str.strip()

    # enforce order
    return out[["trip_id","arrival_time","departure_time","stop_id","stop_sequence","sid","feed"]]

def load_provider(feed, folder):
    stops_path = os.path.join(folder, "stops.txt")
    st_times_path = os.path.join(folder, "stop_times.txt")
    if not os.path.exists(stops_path):     raise FileNotFoundError(f"{feed}: missing {stops_path}")
    if not os.path.exists(st_times_path):  raise FileNotFoundError(f"{feed}: missing {st_times_path}")

    raw_stops = _read_any_sep(stops_path)
    raw_stop_times = _read_any_sep(st_times_path)

    return _clean_stops(raw_stops, feed), _clean_stop_times(raw_stop_times, feed)

# ====== Load, standardize, merge ======
all_stops, all_stop_times = [], []
for feed, folder in PROVIDERS.items():
    try:
        cs, cst = load_provider(feed, folder)
        all_stops.append(cs)
        all_stop_times.append(cst)
        print(f"✅ {feed}: stops {cs.shape}, stop_times {cst.shape}")
    except Exception as e:
        print(f"❌ {feed}: {e}")

stops_merged = pd.concat(all_stops, ignore_index=True, sort=False)
stop_times_merged = pd.concat(all_stop_times, ignore_index=True, sort=False)

# Deduplicate with sid
stops_merged = stops_merged.drop_duplicates(subset=["sid"]).reset_index(drop=True)
stop_times_merged = stop_times_merged.drop_duplicates(subset=["feed","trip_id","sid","stop_sequence"]).reset_index(drop=True)

print("\n=== FINAL ===")
print("stops_merged:", stops_merged.shape, list(stops_merged.columns))
print("stop_times_merged:", stop_times_merged.shape, list(stop_times_merged.columns))

# Quick peek
display(stops_merged.head(10))
display(stop_times_merged.head(10))


✅ carris: stops (12667, 6), stop_times (2433179, 7)
✅ cp: stops (457, 6), stop_times (29469, 7)
✅ fertagus: stops (14, 6), stop_times (3355, 7)
✅ metro: stops (74, 6), stop_times (34041, 7)
✅ mst: stops (19, 6), stop_times (23590, 7)
✅ transportes_barreiro: stops (279, 6), stop_times (60925, 7)
✅ transtejo: stops (9, 6), stop_times (1946, 7)

=== FINAL ===
stops_merged: (13519, 6) ['sid', 'stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'feed']
stop_times_merged: (2586505, 7) ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'sid', 'feed']


,sid,stop_id,stop_name,stop_lat,stop_lon,feed
0,carris_010001,010001,Rua Carlos Manuel Rodrigues Francisco (Escola),38.754244,-8.959557,carris
1,carris_010002,010002,R Carlos M. Francisco 229 (Escola Monte Novo),38.754572,-8.959615,carris
2,carris_010005,010005,ALCOCHETE (R CIPRIÃO FIGUEIREDO),38.754175,-8.961806,carris
3,carris_010007,010007,ALCOCHETE (R LEITE CUNHA) BIBLIOTECA,38.753196,-8.963687,carris
4,carris_010008,010008,ALCOCHETE (R LEITE CUNHA) BIBLIOTECA,38.753271,-8.963504,carris
5,carris_010009,010009,ALCOCHETE (AV RESTAURAÇÃO) EB MANUEL I,38.750706,-8.962749,carris
6,carris_010010,010010,ALCOCHETE (AV RESTAURAÇÃO) EB MANUEL I,38.751002,-8.962783,carris
7,carris_010011,010011,ALCOCHETE (R JOSÉ GRILO EVANGELISTA),38.748855,-8.962159,carris
8,carris_010012,010012,ALCOCHETE (R JOSÉ GRILO EVANGELISTA),38.748866,-8.961929,carris
9,carris_010013,010013,ALCOCHETE (R COOPERAÇÃO 27),38.748733,-8.966155,carris


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,sid,feed
0,4204_0_1|2300|0715_PYPHK,07:33:00,07:33:00,100003,11,carris_100003,carris
1,4204_0_1|2300|0715_PYPHK,07:34:00,07:34:00,100006,12,carris_100006,carris
2,4204_0_1|2300|0715_PYPHK,07:36:00,07:36:00,100008,13,carris_100008,carris
3,4204_0_1|2300|0715_PYPHK,07:37:00,07:37:00,100009,14,carris_100009,carris
4,4204_0_1|2300|0715_PYPHK,07:39:00,07:39:00,100011,15,carris_100011,carris
5,4204_0_1|2300|0715_PYPHK,07:41:00,07:41:00,100027,16,carris_100027,carris
6,4204_0_1|2300|0715_PYPHK,07:41:00,07:41:00,100081,17,carris_100081,carris
7,4204_0_1|2300|0715_PYPHK,07:42:00,07:42:00,100094,18,carris_100094,carris
8,4204_0_1|2300|0715_PYPHK,07:43:00,07:43:00,100120,19,carris_100120,carris
9,4204_0_1|2300|0715_PYPHK,07:44:00,07:44:00,100121,20,carris_100121,carris


In [ ]:
import pandas as pd

stops_merged = pd.read_csv('/content/stops_merged.csv')
stop_times_merged = pd.read_csv('/content/stop_times_merged.csv')

/tmp/ipython-input-2883292311.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times_merged = pd.read_csv('/content/stop_times_merged.csv')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree

# --- constants ---
R = 6371000.0            # Earth radius (m)
WALK_M_PER_MIN = 75.0    # walking speed (m/min)

def gtfs_time_to_s(t: str) -> int:
    h, m, s = map(int, str(t).split(":"))
    return h*3600 + m*60 + s

# === Use the merged tables ===
# Expect: stops_merged columns -> ['sid','stop_id','stop_name','stop_lat','stop_lon','feed']
#         stop_times_merged -> ['trip_id','arrival_time','departure_time','stop_id','stop_sequence','sid','feed']

# Rename once to the generic names this cell expects (lat/lon/name)
stops_min = (
    stops_merged
    .rename(columns={"stop_name":"name","stop_lat":"lat","stop_lon":"lon"})
    .loc[:, ["sid","stop_id","name","feed","lat","lon"]]
    .copy()
)
stops_min["lat"] = pd.to_numeric(stops_min["lat"], errors="coerce")
stops_min["lon"] = pd.to_numeric(stops_min["lon"], errors="coerce")
stops_min = stops_min.dropna(subset=["lat","lon"]).reset_index(drop=True)

# Keep Lisbon/Setúbal (widen if needed)
LIM = dict(lat_min=38.3, lat_max=39.1, lon_min=-9.6, lon_max=-8.4)
stops_min = stops_min[
    stops_min["lat"].between(LIM["lat_min"], LIM["lat_max"])
    & stops_min["lon"].between(LIM["lon_min"], LIM["lon_max"])
].reset_index(drop=True)

# BallTree on ALL stops (for nearest + destination proximity)
all_coords = np.deg2rad(stops_min[["lat","lon"]].to_numpy())
btree_all  = BallTree(all_coords, metric="haversine")

# Per-feed trees
feed_trees, feed_index = {}, {}
for f in stops_min["feed"].dropna().unique():
    df = stops_min[stops_min["feed"] == f].reset_index(drop=True)
    if df.empty:
        continue
    arr = np.deg2rad(df[["lat","lon"]].to_numpy())
    feed_trees[f] = BallTree(arr, metric="haversine")
    feed_index[f] = df

def nearest_stops_from_point(lat, lon, k=6, max_dist_m=1500):
    d_rad, idx = btree_all.query(np.deg2rad([[lat, lon]]), k=min(k, len(stops_min)))
    d_m = d_rad[0] * R
    keep = d_m <= max_dist_m
    if not keep.any():
        best = np.argmin(d_m)
        keep = np.zeros_like(d_m, dtype=bool); keep[best] = True
    idx = idx[0][keep]; d_m = d_m[keep]
    out = stops_min.iloc[idx][["sid","stop_id","name","feed","lat","lon"]].copy()
    out["walk_sec"] = (d_m / WALK_M_PER_MIN) * 60.0
    return out.reset_index(drop=True)

def seeds_per_feed(lat, lon,
                   per_feed_k=5,
                   per_feed_radius_m=5000,
                   take_n_per_feed=2,
                   max_origin_walk_m=None):  # must exist
    out = []
    q = np.deg2rad([[lat, lon]])
    for f, tree in feed_trees.items():
        df = feed_index[f]
        if df.empty:
            continue
        k = min(per_feed_k, len(df))
        d_rad, idx = tree.query(q, k=k)
        d_m = d_rad[0] * R

        # enforce cap per feed
        max_r = per_feed_radius_m if max_origin_walk_m is None else min(per_feed_radius_m, max_origin_walk_m)
        keep = d_m <= max_r
        if not keep.any():
            continue

        idx = idx[0][keep]; d_m = d_m[keep]
        tmp = df.iloc[idx][["sid","stop_id","name","feed","lat","lon"]].copy()
        tmp["walk_sec"] = (d_m / WALK_M_PER_MIN) * 60.0
        tmp = tmp.nsmallest(take_n_per_feed, "walk_sec")
        out.append(tmp)

    if not out:
        return pd.DataFrame(columns=["sid","stop_id","name","feed","lat","lon","walk_sec"])
    return pd.concat(out, ignore_index=True)


In [ ]:
# Cell 3 — Footpaths (walking transfers) — index by sid
# Uses: stops_min, all_coords, btree_all from previous cell

def build_footpaths(radius_m=800):
    if len(stops_min) == 0:
        return {}
    rad = radius_m / R
    inds, dists = btree_all.query_radius(all_coords, r=rad, return_distance=True, sort_results=False)

    ids = stops_min["sid"].to_numpy()
    foot = {}
    for i in range(len(ids)):
        src = ids[i]
        nbr_idx = inds[i]
        nbr_dist_m = dists[i] * R
        walk_sec = (nbr_dist_m / WALK_M_PER_MIN) * 60.0

        rows = []
        for j, dst_idx in enumerate(nbr_idx):
            if int(dst_idx) == i:
                continue
            rows.append((ids[int(dst_idx)], float(walk_sec[j])))
        foot[src] = rows
    return foot

footpaths = build_footpaths(radius_m=600)
print("footpaths built for", len(footpaths), "stops")


footpaths built for 13149 stops


In [ ]:
# Cell 4 — Build connections (namespaced sids)

def build_connections(t_start="05:00:00", t_end="12:00:00"):
    start_s, end_s = gtfs_time_to_s(t_start), gtfs_time_to_s(t_end)

    # keep only stop_times whose stop (sid) is inside the filtered stops_min
    st = stop_times_merged.merge(
        stops_min[["sid","feed"]],
        on=["sid","feed"], how="inner"
    ).copy()

    # times -> seconds
    st["dep_s"] = st["departure_time"].apply(gtfs_time_to_s)
    st["arr_s"] = st["arrival_time"].apply(gtfs_time_to_s)
    st = st[(st["dep_s"].notna()) & (st["arr_s"].notna())]
    st = st[(st["dep_s"] <= end_s) & (st["arr_s"] >= start_s)]

    # next stop within the same trip (per feed)
    st = st.sort_values(["feed","trip_id","stop_sequence"])
    nxt = st.groupby(["feed","trip_id"]).shift(-1)

    # sids already namespaced
    dep_sid = st["sid"]
    arr_sid = nxt["sid"]

    # unique trip id across feeds (optional)
    trip_uid = st["feed"] + ":" + st["trip_id"].astype(str)

    conn = pd.DataFrame({
        "trip_id":  st["trip_id"].values,
        "trip_uid": trip_uid.values,
        "feed":     st["feed"].values,
        "dep_sid":  dep_sid.values,
        "dep_time": st["departure_time"].values,
        "dep_s":    st["dep_s"].astype(int).values,
        "arr_sid":  arr_sid.values,
        "arr_time": nxt["arrival_time"].values,
        "arr_s":    nxt["arr_s"].values,
    }).dropna(subset=["arr_sid","arr_time","arr_s"]).copy()

    conn["arr_s"] = conn["arr_s"].astype(int)
    conn = conn[
        (conn["dep_s"] >= start_s) & (conn["dep_s"] <= end_s) & (conn["arr_s"] >= conn["dep_s"])
    ].sort_values("dep_s").reset_index(drop=True)

    return conn

conn = build_connections("05:00:00", "12:00:00")
print("connections:", conn.shape, "| by feed:\n", conn["feed"].value_counts())


connections: (892547, 9) | by feed:
 feed
carris                  847600
transportes_barreiro     21564
metro                     9448
mst                       7177
cp                        5335
fertagus                  1069
transtejo                  354
Name: count, dtype: int64


In [ ]:
# Cell 5 — Earliest arrival with transfers to a coordinate, using merged tables + sids

from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd

def sec_to_hhmm(s):
    s = int(s)
    h = s // 3600
    m = (s % 3600) // 60
    return f"{h:02d}:{m:02d}"

FEED2MODE = {
    "cp": "train",
    "fertagus": "train",
    "metro": "metro",
    "carris": "bus",
    "mst": "tram",
    "transtejo": "ferry",
    "transportes_barreiro": "bus",
}

def earliest_arrival_to_coord(
    src_lat, src_lon,
    dest_lat, dest_lon,
    depart_after="08:00:00",
    max_dest_walk_m=500,
    max_origin_walk_m=800,
    ride_slack_sec=30,
    walk_slack_sec=30,
    per_feed_k=3,
    per_feed_radius_m=1200,
    take_n_per_feed=2,
    walk_penalty_factor=1.25
):
    # ---- destination stop candidates (within max_dest_walk_m) ----
    d_rad, idx = btree_all.query(np.deg2rad([[dest_lat, dest_lon]]), k=len(stops_min))
    d_m = d_rad[0] * R
    mask = d_m <= max_dest_walk_m
    if not mask.any():
        return {"note": "No stops within destination radius"}
    dest_sids = stops_min.iloc[idx[0][mask]]["sid"].tolist()
    dest_final_walk = {sid: float((m / WALK_M_PER_MIN) * 60.0) for sid, m in zip(dest_sids, d_m[mask])}

    # ---- origin seeds (per feed) ----
    depart_s = gtfs_time_to_s(depart_after)
    seeds = seeds_per_feed(
        src_lat, src_lon,
        per_feed_k=per_feed_k,
        per_feed_radius_m=per_feed_radius_m,
        take_n_per_feed=take_n_per_feed,
        max_origin_walk_m=max_origin_walk_m   # pass the cap
    )
    if seeds.empty:
        return {"note": f"No stops within {max_origin_walk_m} m of origin"}

    INF = 10**12
    arr = pd.Series(INF, index=stops_min["sid"].unique(), dtype="int64")
    parent = {}

    # initial walk to seeds (apply penalty to routing *cost*)
    for _, r in seeds.iterrows():
        sid = r["sid"]
        wsec_real = int(r["walk_sec"])
        wsec_eff  = int(r["walk_sec"] * walk_penalty_factor)
        t0 = depart_s + wsec_eff
        if t0 < arr.get(sid, INF):
            arr[sid] = t0
            parent[sid] = ("walk_origin", None)

    # ---- scan connections ----
    for _, c in conn.iterrows():
        u, v = c["dep_sid"], c["arr_sid"]
        dep, arrv = int(c["dep_s"]), int(c["arr_s"])
        if arr.get(u, INF) + ride_slack_sec <= dep and arrv < arr.get(v, INF):
            arr[v] = arrv
            parent[v] = ("ride", u, c["trip_id"], c["feed"], c["dep_time"], c["arr_time"])
            # walking transfers from v (apply penalty to cost)
            for (w, wsec) in footpaths.get(v, []):
                wsec_eff = int(float(wsec) * walk_penalty_factor)
                cand = arrv + wsec_eff + walk_slack_sec
                if cand < arr.get(w, INF):
                    arr[w] = cand
                    parent[w] = ("walk", v)

    # ---- choose best destination ----
    best_sid, best_total = None, INF
    for sid in dest_sids:
        base = arr.get(sid, INF)
        if base >= INF:
            continue
        total = base + int(dest_final_walk[sid])
        if total < best_total:
            best_total, best_sid = total, sid
    if best_sid is None:
        return {"note": "No path found in time window"}

    # ---- backtrack sids ----
    path = []
    cur = best_sid
    while cur is not None and cur in parent:
        path.append(cur)
        step = parent[cur]
        cur = step[1] if isinstance(step, tuple) and len(step) > 1 else None
    path.reverse()

    sid2name = stops_min.set_index("sid")["name"].to_dict()
    sid2feed = stops_min.set_index("sid")["feed"].to_dict()

    # ---- build legs (walk/ride) ----
    legs = []
    t_cursor = depart_s

    # initial walk (report real, not penalized, duration)
    if path:
        first = path[0]
        if parent[first][0] == "walk_origin":
            w_sec = float(seeds.loc[seeds["sid"] == first, "walk_sec"].min())
            legs.append({
                "type": "walk",
                "from": "Origin",
                "to": sid2name.get(first, first),
                "from_sid": None, "to_sid": first,
                "start": sec_to_hhmm(t_cursor),
                "end":   sec_to_hhmm(t_cursor + w_sec),
                "duration_min": round(w_sec/60.0, 1),
                "meters": round(w_sec/60.0 * WALK_M_PER_MIN, 0),
            })
            t_cursor += w_sec

    # intermediate legs
    for i in range(1, len(path)):
        prev_sid, sid = path[i-1], path[i]
        step = parent[sid]

        if step[0] == "ride":
            u, trip_id, feed, dep_t, arr_t = step[1], step[2], step[3], step[4], step[5]
            dep_s = gtfs_time_to_s(dep_t); arr_s = gtfs_time_to_s(arr_t)
            legs.append({
                "type": "ride",
                "mode": FEED2MODE.get(feed, feed),
                "feed": feed,
                "trip_id": trip_id,
                "from": sid2name.get(prev_sid, prev_sid),
                "to":   sid2name.get(sid, sid),
                "from_sid": prev_sid, "to_sid": sid,
                "dep_time": dep_t, "arr_time": arr_t,
                "duration_min": round((arr_s - dep_s)/60.0, 1),
            })
            t_cursor = arr_s

        elif step[0] == "walk":
            # walk from prev_sid -> sid (report real duration)
            w_sec = None
            for w, wsec in footpaths.get(prev_sid, []):
                if w == sid:
                    w_sec = float(wsec); break
            if w_sec is None:
                a = stops_min.loc[stops_min["sid"]==prev_sid, ["lat","lon"]].values[0]
                b = stops_min.loc[stops_min["sid"]==sid, ["lat","lon"]].values[0]
                d = BallTree.haversine_distances(np.deg2rad([a]), np.deg2rad([b]))[0,0] * R
                w_sec = d / WALK_M_PER_MIN * 60.0
            legs.append({
                "type": "walk",
                "from": sid2name.get(prev_sid, prev_sid),
                "to":   sid2name.get(sid, sid),
                "from_sid": prev_sid, "to_sid": sid,
                "start": sec_to_hhmm(t_cursor),
                "end":   sec_to_hhmm(t_cursor + w_sec),
                "duration_min": round(w_sec/60.0, 1),
                "meters": round(w_sec/60.0 * WALK_M_PER_MIN, 0),
            })
            t_cursor += w_sec

    # final walk to destination coordinate (report real)
    final_w_sec = float(dest_final_walk[best_sid])
    legs.append({
        "type": "walk",
        "from": sid2name.get(best_sid, best_sid),
        "to": "Destination (coords)",
        "from_sid": best_sid, "to_sid": None,
        "start": sec_to_hhmm(t_cursor),
        "end":   sec_to_hhmm(t_cursor + final_w_sec),
        "duration_min": round(final_w_sec/60.0, 1),
        "meters": round(final_w_sec/60.0 * WALK_M_PER_MIN, 0),
    })
    t_cursor += final_w_sec

    return {
        "arrive_time": sec_to_hhmm(best_total),
        "total_min": round((best_total - depart_s)/60.0, 1),
        "dest_sid": best_sid,
        "dest_stop_name": sid2name.get(best_sid, best_sid),
        "final_walk_min": round(final_w_sec/60.0, 1),
        "path_sids": path,
        "path_stop_names": [sid2name.get(s, s) for s in path],
        "legs": legs,
    }



In [ ]:
def print_itinerary(result):
    if "legs" not in result:
        print(result); return
    print(f"Arrive {result['arrive_time']}  (total {result['total_min']} min)")
    for i, leg in enumerate(result["legs"], 1):
        if leg["type"] == "ride":
            mode = leg.get("mode", leg.get("feed", "ride"))
            print(f"{i}. {mode.upper()}: {leg['from']}  →  {leg['to']}   "
                  f"{leg['dep_time']}–{leg['arr_time']}  ({leg['duration_min']} min)")
        else:
            print(f"{i}. WALK: {leg['from']}  →  {leg['to']}   "
                  f"{leg['start']}–{leg['end']}  (~{leg['duration_min']} min, ~{int(leg['meters'])} m)")


In [ ]:
# Cell 6 — Example call

# Listing coordinates (replace with your property)
src_lat, src_lon = 38.7260, -9.1276   # Oeiras example

# Destination coordinates (Marquês de Pombal)
dest_lat, dest_lon = 38.7253, -9.1500

res = earliest_arrival_to_coord(
    src_lat, src_lon,
    dest_lat, dest_lon,
    depart_after="08:00:00",
    max_dest_walk_m=500,
    max_origin_walk_m=1200,    # ← your 1.2 km cap
    ride_slack_sec=30,
    walk_slack_sec=30,
    per_feed_k=3,
    per_feed_radius_m=1200,
    take_n_per_feed=2,
    walk_penalty_factor=1.25
)
print_itinerary(res)


Arrive 08:29  (total 29.5 min)
1. WALK: Origin  →  Anjos   08:00–08:08  (~8.6 min, ~648 m)
2. METRO: Anjos  →  Arroios   08:13:28–08:14:27  (1.0 min)
3. METRO: Arroios  →  Alameda   08:19:52–08:20:40  (0.8 min)
4. WALK: Alameda  →  Alameda   08:20–08:20  (~0.0 min, ~0 m)
5. METRO: Alameda  →  Saldanha   08:21:40–08:23:02  (1.4 min)
6. WALK: Saldanha  →  Saldanha   08:23–08:23  (~0.0 min, ~0 m)
7. METRO: Saldanha  →  Picoas   08:24:54–08:25:50  (0.9 min)
8. METRO: Picoas  →  Marquês de Pombal   08:27:23–08:28:22  (1.0 min)
9. WALK: Marquês de Pombal  →  Destination (coords)   08:28–08:29  (~1.1 min, ~84 m)


Using the model to update a few towns with the time and route to the city center

In [ ]:
df = pd.read_csv('/content/combined_lisbon_setubal.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39740 entries, 0 to 39739
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              39740 non-null  float64
 1   District           39740 non-null  object 
 2   Town               39740 non-null  object 
 3   Type               39740 non-null  object 
 4   TotalArea          39740 non-null  float64
 5   Parking            39740 non-null  int64  
 6   Elevator           39740 non-null  int64  
 7   TotalRooms         39740 non-null  int64  
 8   NumberOfBathrooms  39740 non-null  float64
 9   lat                39740 non-null  float64
 10  lon                39740 non-null  float64
dtypes: float64(5), int64(3), object(3)
memory usage: 3.3+ MB


In [ ]:
df['Town'].unique()

array(['Algueirão-Mem Martins', 'Azeitão (São Lourenço e São Simão)',
       'Misericórdia', 'Ajuda', 'Azambuja', 'Casal de Cambra',
       'Carcavelos e Parede', 'Colares', 'São Domingos de Rana',
       'Alcabideche', 'Alverca do Ribatejo e Sobralinho',
       'Cacém e São Marcos', 'Carvoeira', 'Santo António',
       'Cascais e Estoril', 'Santa Maria Maior',
       'São Domingos de Benfica', 'Odivelas',
       'Santa Iria de Azoia, São João da Talha e Bobadela', 'Loures',
       'Lumiar', 'Massamá e Monte Abraão', 'Vimeiro',
       'Aldeia Galega da Merceana e Aldeia Gavinha', 'Campo de Ourique',
       'Avenidas Novas', 'Carregado e Cadafais', 'Rio de Mouro',
       'Sacavém e Prior Velho', 'Campolide', 'Vialonga', 'Alvalade',
       'Moscavide e Portela',
       'Oeiras e São Julião da Barra, Paço de Arcos e Caxias',
       'Arruda dos Vinhos', 'Encosta do Sol', 'Mafra', 'Beato', 'Arroios',
       'Carnaxide e Queijas', 'Venteira', 'Pontinha e Famões',
       'Agualva e Mira-Sintr

In [ ]:
import pandas as pd

# --- inputs
center_lat, center_lon = 38.7253, -9.1500          # Marquês de Pombal
towns_demo = ["Penha de França", "Massamá e Monte Abraão", "Barreiro e Lavradio"]

demo = df[df["Town"].isin(towns_demo)].copy()      # df already loaded (has Town, lat, lon)

# map sid -> (name, lat, lon) for geometry
_sid2 = stops_min.set_index("sid")[["name","lat","lon"]].to_dict(orient="index")

# 1) Pick ONE representative coordinate per town (median = robust)
town_coords = (
    df[df["Town"].isin(towns_demo)]
      .groupby("Town", as_index=False)[["lat","lon"]]
      .median()
)

# 2) Run the router ONCE per town
def run_router(lat, lon):
    return earliest_arrival_to_coord(
        lat, lon, center_lat, center_lon,
        depart_after="08:00:00",
        max_dest_walk_m=500,
        max_origin_walk_m=1200,
        ride_slack_sec=30, walk_slack_sec=30,
        per_feed_k=3, per_feed_radius_m=1200, take_n_per_feed=2,
        walk_penalty_factor=1.25
    )

def legs_to_text(legs):
    parts = []
    for L in legs:
        if L["type"] == "ride":
            mode = L.get("mode", L.get("feed","RIDE")).upper()
            parts.append(f'{mode}: {L["from"]}→{L["to"]} {L["dep_time"]}-{L["arr_time"]}')
        else:
            parts.append(f'WALK: {L["from"]}→{L["to"]} {L["start"]}-{L["end"]} (~{L["meters"]} m)')
    return " | ".join(parts)

# run once per town
rows = []
for _, r in town_coords.iterrows():
    res = run_router(r.lat, r.lon)
    if "legs" in res:
        rows.append({
            "Town": r.Town,
            "travel_min": res["total_min"],
            "arrive_time": res["arrive_time"],
            "route_text": legs_to_text(res["legs"])
        })
    else:
        rows.append({
            "Town": r.Town,
            "travel_min": None,
            "arrive_time": None,
            "route_text": res.get("note","no route")
        })

town_routes = pd.DataFrame(rows)

def legs_to_points(town, lat0, lon0, res):
    pts, seq = [], 0
    pts.append((town, seq, float(lat0), float(lon0), "origin")); seq += 1
    for leg in res.get("legs", []):
        to_sid = leg.get("to_sid")
        if to_sid and to_sid in _sid2:
            pts.append((town, seq, _sid2[to_sid]["lat"], _sid2[to_sid]["lon"], leg["type"]))
            seq += 1
    pts.append((town, seq, center_lat, center_lon, "destination"))
    return pts

route_pts = []
for _, r in town_coords.iterrows():
    res = run_router(r.lat, r.lon)
    if "legs" in res:
        route_pts += legs_to_points(r.Town, r.lat, r.lon, res)

# 1) Save the per-town summary (1 row per town)
town_routes.to_csv("/content/demo_town_routes_summary.csv", index=False)

# 2) Save the full housing dataset joined with those times (many rows)
df_out = df.merge(town_routes, on="Town", how="left")
df_out.to_csv("/content/housing_with_citycenter_times.csv", index=False)

points_out = pd.DataFrame(route_pts, columns=["Town","PathOrder","lat","lon","segment_type"])
demo_out   = town_routes.copy()



# 3) Save the route points table (for lines on map)
points_out.to_csv("/content/demo_town_routes_points.csv", index=False)

# Preview
demo_out.head(10), points_out.head(10)


(                     Town  travel_min arrive_time  \
 0     Barreiro e Lavradio        57.8       08:57   
 1  Massamá e Monte Abraão        49.6       08:49   
 2         Penha de França        29.5       08:29   
 
                                           route_text  
 0  WALK: Origin→Lavradio 08:00-08:09 (~745.0 m) |...  
 1  WALK: Origin→Monte Abraao 08:00-08:10 (~789.0 ...  
 2  WALK: Origin→Anjos 08:00-08:09 (~706.0 m) | ME...  ,
                   Town  PathOrder        lat       lon segment_type
 0  Barreiro e Lavradio          0  38.668757 -9.058565       origin
 1  Barreiro e Lavradio          1  38.662091 -9.059446         walk
 2  Barreiro e Lavradio          2  38.659875 -9.071594         ride
 3  Barreiro e Lavradio          3  38.653656 -9.080153         ride
 4  Barreiro e Lavradio          4  38.651800 -9.078400         walk
 5  Barreiro e Lavradio          5  38.707000 -9.133600         ride
 6  Barreiro e Lavradio          6  38.707207 -9.133351         walk
 7  B

I used my model for 3 of my towns, due to some limitations lets try to use google API's to figure out the rest of the towns

In [ ]:
# --- CONFIG ---
GOOGLE_API_KEY = "paste_you_key"   # ← for privacy purposes I have hidden the key
CENTER = (38.7253, -9.1500)            # Marquês de Pombal
MODE = "transit"                       # 'transit' | 'driving' | 'walking' | 'bicycling'


towns_demo = ["Penha de França", "Massamá e Monte Abraão", "Barreiro e Lavradio"]

# Use df_out if it exists, otherwise fall back to df
try:
    _base = df_out.copy()
except NameError:
    _base = df.copy()

# sanity
assert {"Town","lat","lon"} <= set(_base.columns), "df_out/df must have Town, lat, lon"

# unique towns NOT in demo, with a robust representative coordinate (median)
other_towns = (
    _base.loc[~_base["Town"].isin(towns_demo), ["Town","lat","lon"]]
         .dropna(subset=["Town","lat","lon"])
         .groupby("Town", as_index=False)[["lat","lon"]]
         .median()
)

print("Unique towns to query (Google):", len(other_towns))
other_towns.head()


Unique towns to query (Google): 184


,Town,lat,lon
0,A dos Cunhados e Maceira,39.157597,-9.320485
1,Abela,38.000742,-8.558629
2,Abrigada e Cabanas de Torres,39.144693,-9.031316
3,Agualva e Mira-Sintra,38.773876,-9.295235
4,Ajuda,38.704676,-9.199604


In [ ]:
!pip -q install googlemaps
import googlemaps, pandas as pd, time
from datetime import datetime

gmaps = googlemaps.Client(key=GOOGLE_API_KEY)


  Preparing metadata (setup.py) ... done


In [ ]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo   # Python 3.9+

# 1) Next weekday (Mon–Fri) at 08:00 in Lisbon
def next_weekday_at(hour=8, minute=0, tz="Europe/Lisbon"):
    now = datetime.now(ZoneInfo(tz))
    target = now.replace(hour=hour, minute=minute, second=0, microsecond=0)
    # move to tomorrow until it's Mon–Fri and strictly in the future
    while target <= now or target.weekday() >= 5:
        target = (target + timedelta(days=1)).replace(hour=hour, minute=minute,
                                                     second=0, microsecond=0)
    return target

DEPART_AT = next_weekday_at(8, 0, "Europe/Lisbon")

In [ ]:
def sec_to_hhmm(s:int) -> str:
    s = int(s)
    h = s // 3600
    m = (s % 3600) // 60
    return f"{h:02d}:{m:02d}"

def _fmt_epoch_hhmm(x):
    try:
        return datetime.fromtimestamp(int(x)).strftime("%H:%M")
    except Exception:
        return None

def build_route_text(steps):
    parts = []
    for st in steps:
        mode = (st.get("travel_mode") or "").upper()
        if mode == "TRANSIT":
            td = st.get("transit_details") or {}
            dep = (td.get("departure_stop") or {}).get("name", "Unknown")
            arr = (td.get("arrival_stop") or {}).get("name", "Unknown")
            line = (td.get("line") or {}).get("short_name") or (td.get("line") or {}).get("name") or "Transit"
            dep_t = _fmt_epoch_hhmm((td.get("departure_time") or {}).get("value"))
            arr_t = _fmt_epoch_hhmm((td.get("arrival_time") or {}).get("value"))
            parts.append(f"TRANSIT({line}): {dep}→{arr} {dep_t}-{arr_t}")
        elif mode == "WALKING":
            dur_s = (st.get("duration") or {}).get("value", 0)
            dist_m = (st.get("distance") or {}).get("value", 0)
            parts.append(f"WALK: {sec_to_hhmm(dur_s)} (~{int(dist_m)} m)")
        else:
            dur_s = (st.get("duration") or {}).get("value", 0)
            dist_m = (st.get("distance") or {}).get("value", 0)
            parts.append(f"{mode or 'STEP'}: {sec_to_hhmm(dur_s)} (~{int(dist_m)} m)")
    return " | ".join(parts)

def step_points_with_type(step):
    """Decode each step's polyline and tag with segment type (walk/ride/etc)."""
    from googlemaps.convert import decode_polyline
    pts = []
    mode = (step.get("travel_mode") or "").upper()
    seg = "ride" if mode == "TRANSIT" else ("walk" if mode == "WALKING" else mode.lower() or "other")
    poly = (step.get("polyline") or {}).get("points")
    if poly:
        for p in decode_polyline(poly):
            pts.append((p["lat"], p["lng"], seg))
    else:
        end = step.get("end_location")
        if end:
            pts.append((end["lat"], end["lng"], seg))
    return pts


In [ ]:
def google_route_for_town(lat: float, lon: float):
    resp = gmaps.directions(
        origin=(float(lat), float(lon)),
        destination=CENTER,
        mode="transit",
        departure_time=DEPART_AT
    )
    if not resp:
        return None

    route = resp[0]
    leg = route["legs"][0]

    travel_min = round(leg["duration"]["value"] / 60, 1)
    arrive_time = (leg.get("arrival_time") or {}).get("text")
    if not arrive_time:
        dep_epoch = (leg.get("departure_time") or {}).get("value")
        if dep_epoch:
            arrive_time = _fmt_epoch_hhmm(int(dep_epoch) + int(leg["duration"]["value"]))

    rtext = build_route_text(leg.get("steps", []))

    seq = 0
    pts = [(float(lat), float(lon), "origin", seq)]; seq += 1
    for st in leg.get("steps", []):
        for plat, plon, seg in step_points_with_type(st):
            pts.append((plat, plon, seg, seq)); seq += 1
    pts.append((CENTER[0], CENTER[1], "destination", seq)); seq += 1

    return {"travel_min": travel_min, "arrive_time": arrive_time, "route_text": rtext, "points": pts}



In [ ]:
rows = []
all_points = []

for _, r in other_towns.iterrows():
    out = google_route_for_town(r["lat"], r["lon"])
    time.sleep(0.15)  # mild throttle
    if not out:
        rows.append({"Town": r["Town"], "travel_min_google": None, "arrive_time_google": None, "route_text_google": "no route"})
        continue
    rows.append({
        "Town": r["Town"],
        "travel_min_google": out["travel_min"],
        "arrive_time_google": out["arrive_time"],
        "route_text_google": out["route_text"]
    })
    # store points
    all_points += [(r["Town"], seq, lat, lon, seg) for (lat, lon, seg, seq) in out["points"]]

google_summary = pd.DataFrame(rows)
google_points  = pd.DataFrame(all_points, columns=["Town","PathOrder","lat","lon","segment_type"])

google_summary.head(), google_points.head()


(                           Town  travel_min_google arrive_time_google  \
 0      A dos Cunhados e Maceira                NaN               None   
 1                         Abela                NaN               None   
 2  Abrigada e Cabanas de Torres                NaN               None   
 3         Agualva e Mira-Sintra               55.1            8:58 AM   
 4                         Ajuda               38.6            8:46 AM   
 
                                    route_text_google  
 0                                           no route  
 1                                           no route  
 2                                           no route  
 3  WALK: 00:15 (~1143 m) | TRANSIT(Sintra): Agual...  
 4  WALK: 00:02 (~142 m) | TRANSIT(771): R. Bica M...  ,
                     Town  PathOrder        lat       lon segment_type
 0  Agualva e Mira-Sintra          0  38.773876 -9.295235       origin
 1  Agualva e Mira-Sintra          1  38.773880 -9.295350         walk
 2  

In [ ]:
# towns without a transit route from the previous step
no_route = google_summary[google_summary["travel_min_google"].isna()][["Town"]].merge(
    other_towns, on="Town", how="left"
)
print(f"Missing transit routes: {len(no_route)}")
no_route.head()



Missing transit routes: 38


,Town,lat,lon
0,A dos Cunhados e Maceira,39.157597,-9.320485
1,Abela,38.000742,-8.558629
2,Abrigada e Cabanas de Torres,39.144693,-9.031316
3,Alcácer do Sal (Santa Maria do Castelo e Santi...,38.396486,-8.498626
4,Aldeia Galega da Merceana e Aldeia Gavinha,39.095490,-9.118322


In [ ]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import time

# Get 08:00 on next weekday (Mon–Fri) in Lisbon
def next_weekday_at(hour=8, minute=0, tz="Europe/Lisbon"):
    now = datetime.now(ZoneInfo(tz))
    target = now.replace(hour=hour, minute=minute, second=0, microsecond=0)
    while target <= now or target.weekday() >= 5:
        target = (target + timedelta(days=1)).replace(hour=hour, minute=minute,
                                                     second=0, microsecond=0)
    return target

DEPART_AT = next_weekday_at(8, 0, "Europe/Lisbon")

drive_rows = []
for _, r in other_towns.iterrows():
    origins = [(float(r["lat"]), float(r["lon"]))]
    destinations = [CENTER]
    try:
        dm = gmaps.distance_matrix(
            origins=origins,
            destinations=destinations,
            mode="driving",
            departure_time=DEPART_AT,
        )
        el = dm["rows"][0]["elements"][0]
        if el.get("status") == "OK":
            dist_m = el["distance"]["value"]
            dur_s = el.get("duration_in_traffic", el["duration"])["value"]
            drive_rows.append({
                "Town": r["Town"],
                "drive_min": round(dur_s / 60, 1),
                "drive_km": round(dist_m / 1000, 1)
            })
        else:
            drive_rows.append({"Town": r["Town"], "drive_min": None, "drive_km": None})
    except:
        drive_rows.append({"Town": r["Town"], "drive_min": None, "drive_km": None})
    time.sleep(0.1)  # gentle throttle

driving_df = pd.DataFrame(drive_rows)
driving_df.head()



,Town,drive_min,drive_km
0,A dos Cunhados e Maceira,48.6,62.3
1,Abela,93.6,151.3
2,Abrigada e Cabanas de Torres,52.7,58.5
3,Agualva e Mira-Sintra,19.6,21.4
4,Ajuda,12.0,7.1


In [ ]:
# consolidate duplicate driving columns created by previous merges
cols = google_summary.columns

def coalesce_pair(df, base):
    x, y = f"{base}_x", f"{base}_y"
    if x in df.columns or y in df.columns:
        df[base] = df[[c for c in [x, y] if c in df.columns]].bfill(axis=1).iloc[:, 0]
        df.drop(columns=[c for c in [x, y] if c in df.columns], inplace=True, errors="ignore")

coalesce_pair(google_summary, "drive_min")
coalesce_pair(google_summary, "drive_km")

# sanity check
google_summary[["Town","travel_min_google","drive_min","drive_km"]].head()


,Town,travel_min_google,drive_min,drive_km
0,A dos Cunhados e Maceira,NaN,48.6,62.3
1,Abela,NaN,93.6,151.3
2,Abrigada e Cabanas de Torres,NaN,52.7,58.5
3,Agualva e Mira-Sintra,55.1,19.6,21.4
4,Ajuda,38.6,12.0,7.1


In [ ]:
google_summary.loc[google_summary["travel_min_google"].isna(),
                   ["Town","drive_min","drive_km"]].to_csv(
    "/content/towns_no_transit_driving_fallback.csv", index=False
)


In [ ]:
google_summary

/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Town,travel_min_google,arrive_time_google,route_text_google,drive_min,drive_km
0,A dos Cunhados e Maceira,NaN,None,no route,48.6,62.3
1,Abela,NaN,None,no route,93.6,151.3
2,Abrigada e Cabanas de Torres,NaN,None,no route,52.7,58.5
3,Agualva e Mira-Sintra,55.1,8:58 AM,WALK: 00:15 (~1143 m) | TRANSIT(Sintra): Agual...,19.6,21.4
4,Ajuda,38.6,8:46 AM,WALK: 00:02 (~142 m) | TRANSIT(771): R. Bica M...,12.0,7.1
...,...,...,...,...,...,...
179,Vila Nova da Rainha,67.6,9:17 AM,WALK: 00:02 (~157 m) | TRANSIT(Azambuja): Vila...,53.4,51.9
180,Vila Verde dos Francos,NaN,None,no route,58.6,75.7
181,Vilar,273.3,12:52 PM,WALK: 00:24 (~1755 m) | TRANSIT(B): Vilar do P...,200.5,330.4
182,Vimeiro,152.4,10:39 AM,WALK: 00:23 (~1618 m) | TRANSIT(7232): Silval ...,79.7,114.4


In [ ]:
# town_routes already exists from your model run (3 towns)
# Get their coords (median per town from your df_out)
model_towns_coords = (
    df_out[df_out["Town"].isin(town_routes["Town"])]
      .groupby("Town", as_index=False)[["lat","lon"]].median()
)

drive_rows = []
for _, r in model_towns_coords.iterrows():
    try:
        dm = gmaps.distance_matrix(
            origins=[(r["lat"], r["lon"])],
            destinations=[CENTER],
            mode="driving",
            departure_time=DEPART_AT
        )
        el = dm["rows"][0]["elements"][0]
        if el.get("status") == "OK":
            dist_m = el["distance"]["value"]
            dur_s = el.get("duration_in_traffic", el["duration"])["value"]
            drive_rows.append({
                "Town": r["Town"],
                "drive_min": round(dur_s/60, 1),
                "drive_km": round(dist_m/1000, 1)
            })
        else:
            drive_rows.append({"Town": r["Town"], "drive_min": None, "drive_km": None})
    except:
        drive_rows.append({"Town": r["Town"], "drive_min": None, "drive_km": None})

model_driving_df = pd.DataFrame(drive_rows)
model_driving_df


,Town,drive_min,drive_km
0,Barreiro e Lavradio,33.6,37.7
1,Massamá e Monte Abraão,18.2,14.4
2,Penha de França,11.9,4.1


In [ ]:
town_routes = town_routes.merge(model_driving_df, on="Town", how="left")
town_routes


,Town,travel_min,arrive_time,route_text,drive_min,drive_km
0,Barreiro e Lavradio,57.8,08:57,WALK: Origin→Lavradio 08:00-08:09 (~745.0 m) |...,33.6,37.7
1,Massamá e Monte Abraão,49.6,08:49,WALK: Origin→Monte Abraao 08:00-08:10 (~789.0 ...,18.2,14.4
2,Penha de França,29.5,08:29,WALK: Origin→Anjos 08:00-08:09 (~706.0 m) | ME...,11.9,4.1


In [ ]:
# ensure google_summary has correct driving columns
for c in ["drive_min","drive_km"]:
    if c not in google_summary.columns:
        google_summary[c] = None

# unify columns to match
google_summary.rename(columns={
    "travel_min_google": "travel_min",
    "arrive_time_google": "arrive_time",
    "route_text_google": "route_text"
}, inplace=True)

all_towns_summary = pd.concat([town_routes, google_summary], ignore_index=True)

# save
all_towns_summary.to_csv("/content/all_towns_summary.csv", index=False)

all_towns_summary.sample(5)


,Town,travel_min,arrive_time,route_text,drive_min,drive_km
13,Aldeia Galega da Merceana e Aldeia Gavinha,NaN,None,no route,55.4,66.2
122,Pontinha e Famões,37.5,8:41 AM,WALK: 00:00 (~57 m) | TRANSIT(2814): R Major J...,18.6,15.7
185,Vimeiro,152.4,10:39 AM,WALK: 00:23 (~1618 m) | TRANSIT(7232): Silval ...,79.7,114.4
62,Casal de Cambra,48.6,8:54 AM,WALK: 00:04 (~292 m) | TRANSIT(1709): Av Dinam...,21.1,18.6
158,"Sintra (Santa Maria e São Miguel, São Martinho...",62.0,9:07 AM,WALK: 00:04 (~319 m) | TRANSIT(1248): Av Aviaç...,26.6,31.7


In [ ]:
all_towns_summary['Town'].nunique()

187

In [ ]:
# all_towns_summary must have: Town, travel_min, arrive_time, route_text, drive_min, drive_km
summary = (
    all_towns_summary
      .loc[:, ["Town","travel_min","arrive_time","route_text","drive_min","drive_km"]]
      .dropna(subset=["Town"])
      .drop_duplicates(subset=["Town"])
      .reset_index(drop=True)
)
summary.head()


,Town,travel_min,arrive_time,route_text,drive_min,drive_km
0,Barreiro e Lavradio,57.8,08:57,WALK: Origin→Lavradio 08:00-08:09 (~745.0 m) |...,33.6,37.7
1,Massamá e Monte Abraão,49.6,08:49,WALK: Origin→Monte Abraao 08:00-08:10 (~789.0 ...,18.2,14.4
2,Penha de França,29.5,08:29,WALK: Origin→Anjos 08:00-08:09 (~706.0 m) | ME...,11.9,4.1
3,A dos Cunhados e Maceira,NaN,None,no route,48.6,62.3
4,Abela,NaN,None,no route,93.6,151.3


In [ ]:
# --- Defensive merge of per-town summary into the main table ---

# summary must have: Town, travel_min, arrive_time, route_text, drive_min, drive_km
need_cols = {"Town","travel_min","arrive_time","route_text","drive_min","drive_km"}
for c in need_cols - set(all_towns_summary.columns):
    all_towns_summary[c] = pd.NA

summary = (
    all_towns_summary
      .loc[:, ["Town","travel_min","arrive_time","route_text","drive_min","drive_km"]]
      .dropna(subset=["Town"])
      .drop_duplicates(subset=["Town"])
      .reset_index(drop=True)
)

base = df_out.copy()

# merge with suffixes to avoid collisions
merged = base.merge(summary, on="Town", how="left", suffixes=("", "_sum"))

def make_final(df, name):
    """Create <name>_final from existing <name> and/or <name>_sum if either exists."""
    src_old = name if name in df.columns else None
    src_new = f"{name}_sum" if f"{name}_sum" in df.columns else None
    if src_old and src_new:
        df[f"{name}_final"] = df[src_old].combine_first(df[src_new])
    elif src_old:
        df[f"{name}_final"] = df[src_old]
    elif src_new:
        df[f"{name}_final"] = df[src_new]
    # if neither exists, do nothing
    return df

for col in ["travel_min", "arrive_time", "route_text", "drive_min", "drive_km"]:
    merged = make_final(merged, col)

# drop the raw/suffixed columns, keep only *_final + original non-route fields
drop_cols = [c for c in ["travel_min","arrive_time","route_text","drive_min","drive_km",
                         "travel_min_sum","arrive_time_sum","route_text_sum",
                         "drive_min_sum","drive_km_sum"] if c in merged.columns]
merged = merged.drop(columns=drop_cols)

# save and preview (pick only columns that exist)
merged.to_csv("/content/housing_with_citycenter_times_final.csv", index=False)

show_cols = ["Town","travel_min_final","arrive_time_final",
             "route_text_final","drive_min_final","drive_km_final"]
print(merged[ [c for c in show_cols if c in merged.columns] ].head(10))



                                 Town  travel_min_final arrive_time_final  \
0               Algueirão-Mem Martins              52.9           8:58 AM   
1  Azeitão (São Lourenço e São Simão)              73.3           9:26 AM   
2                        Misericórdia              18.9           8:21 AM   
3                               Ajuda              38.6           8:46 AM   
4                            Azambuja              76.4           9:17 AM   
5                               Ajuda              38.6           8:46 AM   
6                     Casal de Cambra              48.6           8:54 AM   
7                               Ajuda              38.6           8:46 AM   
8                               Ajuda              38.6           8:46 AM   
9                               Ajuda              38.6           8:46 AM   

                                    route_text_final  drive_min_final  \
0  WALK: 00:05 (~401 m) | TRANSIT(Sintra): Alguei...             25.1   
1  WAL

Lets make sure our final data is Clean and start build our machine learning model

In [ ]:
import pandas as pd

# Load your final Excel file
df = pd.read_excel("/content/housing_with_citycenter_times_final.xlsx")

# Check for duplicates (all columns)
dupes = df[df.duplicated()]
print(f"Found {len(dupes)} duplicate rows")

# Remove them
df = df.drop_duplicates().reset_index(drop=True)


Found 4059 duplicate rows


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30222 entries, 0 to 30221
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              30222 non-null  float64
 1   District           30222 non-null  object 
 2   Town               30222 non-null  object 
 3   Type               30222 non-null  object 
 4   TotalArea          30222 non-null  int64  
 5   Parking            30222 non-null  int64  
 6   Elevator           30222 non-null  int64  
 7   TotalRooms         30222 non-null  int64  
 8   NumberOfBathrooms  30222 non-null  int64  
 9   lat                30222 non-null  float64
 10  lon                30222 non-null  float64
 11  travel_min_final   28415 non-null  float64
 12  arrive_time_final  28415 non-null  object 
 13  route_text_final   30222 non-null  object 
 14  drive_min_final    30222 non-null  float64
 15  drive_km_final     30222 non-null  float64
dtypes: float64(6), int64(5

In [ ]:
df["price_per_m2"] = df["Price"] / df["TotalArea"]
df["price_per_m2"].head()


,price_per_m2
0,2531.645570
1,329.457364
2,5409.090909
3,4632.352941
4,480.769231


Solving the blanks of towns that dont have a transportation route, lets create a flag

In [ ]:
df["no_transit_route"] = df["travel_min_final"].isna().astype(int)
#1 → no public route found
#0 → route exists


In [ ]:
df["travel_min_final"] = df["travel_min_final"].fillna(df["drive_min_final"])


Lets try Linear Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# ---- features/target
features = [
    "District","Town","Type",
    "TotalArea","TotalRooms","NumberOfBathrooms","Parking","Elevator",
    "travel_min_final","drive_min_final","drive_km_final","no_transit_route"
  ]
target = "price_per_m2"

X = df[features].copy()
y = df[target].copy()

# ---- preprocessors WITH IMPUTATION
cat_cols = ["District","Town","Type"]
num_cols = [c for c in features if c not in cat_cols]

preprocessor = ColumnTransformer([
    ("cat",
     Pipeline([
         ("imputer", SimpleImputer(strategy="most_frequent")),
         ("onehot", OneHotEncoder(handle_unknown="ignore"))
     ]),
     cat_cols),
    ("num",
     Pipeline([
         ("imputer", SimpleImputer(strategy="median"))
     ]),
     num_cols)
])

# ---- split, model, fit
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = Pipeline([
    ("prep", preprocessor),
    ("reg", LinearRegression())
])

model.fit(X_train, y_train)

# ---- evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.3f}")


RMSE: 2328.08
R²:   0.170


We got a really bad score with linear Regression, the data probably its really messy. Lets compare our model with a baseline model

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Dummy model: always predicts the mean of y_train
dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train, y_train)

# Predictions
y_pred_dummy = dummy.predict(X_test)

# Metrics
rmse_dummy = np.sqrt(mean_squared_error(y_test, y_pred_dummy))
r2_dummy = r2_score(y_test, y_pred_dummy)

print(f"Dummy RMSE: {rmse_dummy:.2f}")
print(f"Dummy R²: {r2_dummy:.3f}")


Dummy RMSE: 2556.13
Dummy R²: -0.000


The Linear Regression Model did better than our baseline model that just predicts the mean, but 17% of predicting the variance is still really low. Lets try another model

In [ ]:
import numpy as np
import pandas as pd

# target
y = df["price_per_m2"].copy()

# features (drop 'Town' to avoid high-cardinality noise)
features = [
    "Town","Type",
    "TotalArea","TotalRooms","NumberOfBathrooms","Parking","Elevator",
    "travel_min_final","drive_min_final","drive_km_final",
    "no_transit_route"  # keep this if you created it earlier
]
X = df[features].copy()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

cat_cols = ["Town","Type"]
num_cols = [c for c in features if c not in cat_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols),
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median"))
        ]), num_cols),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

gbr = HistGradientBoostingRegressor(
    learning_rate=0.07,
    max_leaf_nodes=31,
    min_samples_leaf=20,
    l2_regularization=1.0,
    early_stopping="auto",
    random_state=42,
)

gbr_pipe = Pipeline([
    ("prep", preprocessor),
    ("reg", gbr),
])

gbr_pipe.fit(X_train, y_train)

y_pred = gbr_pipe.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"GBR RMSE: {rmse:.2f}")
print(f"GBR R²:   {r2:.3f}")


GBR RMSE: 1299.65
GBR R²:   0.741


Lets try randomforest

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# === features/target (same as your GBR run) ===
features = [
    "Town","Type",
    "TotalArea","TotalRooms","NumberOfBathrooms","Parking","Elevator",
    "travel_min_final","drive_min_final","drive_km_final",
    "no_transit_route"
]
X = df[features].copy()
y = df["price_per_m2"].copy()

cat_cols = ["Town","Type"]            # you said you’re using Town
num_cols = [c for c in features if c not in cat_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols),
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median"))
        ]), num_cols),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rf_pipe = Pipeline([
    ("prep", preprocessor),
    ("reg", rf),
])

rf_pipe.fit(X_train, y_train)

y_pred = rf_pipe.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RF RMSE: {rmse:.2f}")
print(f"RF R²:   {r2:.3f}")


RF RMSE: 1281.67
RF R²:   0.749


Lets save our machine learning model to use on Streamlit

In [ ]:
import joblib, json

# use the same feature list you trained with
features = [
    "Town","Type",
    "TotalArea","TotalRooms","NumberOfBathrooms","Parking","Elevator",
    "travel_min_final","drive_min_final","drive_km_final",
    "no_transit_route"
]

joblib.dump(rf_pipe, "/content/rf_price_per_m2.joblib")
with open("/content/rf_price_features.json","w") as f:
    json.dump(features, f)

print("Saved: /content/rf_price_per_m2.joblib and rf_price_features.json")


Saved: /content/rf_price_per_m2.joblib and rf_price_features.json
